---

# Exploratory Data Analysis : <font color='tomato'>Give me Categorical material </font>

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns",None)

path_input='./../data/data_raw/'
path_output='./../data/data_created/' 

### input and output du notebook
- Input
    * Request.csv
    * OperatingSpecialty.csv
    * BodyLocation_modif.csv
    * Laboratory.csv
- Ouput
    * SpecialtyAndCity3.csv
    * Operation_Material_Rq_Final.csv
    * Operation_Material_Rq.csv
    * MainOperation.csv
    * Operation_Material_Rq_Final.csv
    

## Import 
### table request
#### df_request

In [ ]:
df_request = pd.read_csv(path_input+"Request.csv", low_memory=False)
df_request.head(2)

In [ ]:
df_request['Used Implants'].value_counts()

#### Choix des colonnes table request

In [ ]:
df_rq = df_request[["ID","Expected Material", 'Chosen Laboratory ID',
                    "Location ID", "Operating Block User ID","Specialty ID","Surgeon"]]

In [ ]:
mask = df_request['Specialty ID'].isna()
df_nan = df_request.loc[mask,:]
df_nan.shape

#### Est-ce qu'il y a des NaN ?

In [ ]:
df_rq.isnull().sum().sort_values(ascending=False)

In [ ]:
df_nan.head(2)

---

### Import table Operating Specialty (les DMI)
#### df_SpeOpe

In [ ]:
df_SpeOpe = pd.read_csv(path_input+"OperatingSpecialty.csv")
df_SpeOpe = df_SpeOpe.set_index("ID")
df_SpeOpe.head(2)

---

### `df_Spe` : Merge des specialités opérations & request

In [ ]:
df_Spe = pd.merge(df_rq, df_SpeOpe, left_on = "Specialty ID", right_on = "ID" )
df_Spe.head(3)

In [ ]:
df_Spe.shape

#### `df_SplitMaterial`: Aggrégation : nb de requêtes par DMI : 

In [ ]:
df_SpeMaterial = df_Spe.groupby('Name').count().reset_index()
df_SpeMaterial.head(2)

In [ ]:
df_SplitMaterial = df_SpeMaterial[["Name","ID"]]
df_SplitMaterial = df_SplitMaterial.rename(columns = {"Name":"Categorical_Material","ID" : "Nb_requests"})
df_SplitMaterial = df_SplitMaterial.sort_values('Nb_requests',ascending=False)
df_SplitMaterial.head(2)

In [ ]:
df_SplitMaterialGraph = df_SplitMaterial.set_index("Categorical_Material")
df_SplitMaterialGraph.head(3)

#### Plot request per Categorical material

In [ ]:
plt.figure(figsize=(10,5))
df_SplitMaterialGraph['Nb_requests'].plot(kind = "bar",legend = False)

plt.title("Number of request per Categorical material")

#xticks 
plt.xticks(rotation=70) 
#x-axis labels 
plt.xlabel('Categorical material') 
#y-axis labels 

#save plot 
plt.savefig('plot.png',dpi=300,bbox_inches='tight') 
#display 

plt.show()


In [ ]:
plot = df_SplitMaterialGraph.plot.pie(y='Nb_requests', figsize=(5, 5))

---

###  `df_operation`n Merge avec body location : `df_bodyloc` & `df_Spe`
Objectif => avoir le nb de requets par segment anatomique + matériel 
#### Check `df_Spe`

In [ ]:
df_Spe.head(2)

#### Read BodyLocation

In [ ]:
df_bodyloc = pd.read_csv(path_input+"BodyLocation_modif.csv")
df_bodyloc.head(2)

#### `df_operation` Merge df_spe & df_bodyloc

In [ ]:
df_operation1 = pd.merge(df_Spe, df_bodyloc, left_on = "Location ID", right_on = "ID" )
df_operation1.head(2)


In [ ]:
df_operation2 = pd.merge(df_Spe, df_bodyloc, left_on = "Location ID", right_on = "ID" )
df_operation2.head(2)

### `df_operationMore` 
#### : renommer les colonnes

In [ ]:
df_operationMore = df_operation1[["ID_x","Name_x","SegmentAnat"]]
df_operationMore = df_operation1.rename(columns = {"ID_x":"Id_rq","Name_x" : "DMI"})
df_operationMore.head()
#df_operation = df_operation.rename(columns = {"ID_x":"Id_rq","Name_x" : "DMI"})

---

### `df_labo` :Upload table labo 
#### Read laboratory.csv

In [ ]:
df_labo = pd.read_csv(path_input+"Laboratory.csv")

In [ ]:
df_labo.shape

In [ ]:
df_labo.head(3)

In [ ]:
df_labo.isnull().sum().sort_values(ascending=False)

#### Choix des colonnes 

In [ ]:
df_labo = df_labo[["Name","ID","Zip Code","City"]]
df_labo.head(3)

In [ ]:
df_labo = df_labo[df_labo['Name'] != "TEST"]
df_labo.head(2)

### `df_operationWithLabo`  :Merge labo & operation : 

In [ ]:
df_operation1.head(2)

In [ ]:
df_operationWithLabo = pd.merge(df_operation1, df_labo, left_on ="Chosen Laboratory ID",right_on="ID")
df_operationWithLabo.head(2)

In [ ]:
df_materialLabo = df_operationWithLabo [["ID_x","Name_x","SegmentAnat","Name","City","Zip Code"]]
df_materialLabo.sample(2)

In [ ]:
df_materialLabo.shape

#### OUTPUT `SpecialtyAndCity3.csv`

In [ ]:
#Dans le fichier .csv sur excel > j'ai rajouté une colonne département pour pouvoir localiser les labos sur Tableau
df_materialLabo.to_csv(path_output+"SpecialtyAndCity3.csv")

#### Exploration df_materialLabo

In [ ]:
df1 = df_materialLabo.groupby('Name').agg('count')

In [ ]:
df_materialLabo[df_materialLabo['Name']=='non désigné']

In [ ]:
df_materialLabo.isnull().sum().sort_values(ascending=False)

---

In [ ]:
df_operation1.head(2)

In [ ]:
df_operationMore.head(2)

In [ ]:
df_MaterialOperation = df_operationMore[["Id_rq","DMI","SegmentAnat"]]
df_MaterialOperation.sample(2)
                    

In [ ]:
df_MaterialOperation.shape

#### OUTPOUT `Operation_Material_Rq_Final.csv`

In [ ]:
df_MaterialOperation.to_csv(path_output+"Operation_Material_Rq_Final.csv")

---

### Isoler 1ère intention et reprise : `df_FirstIntention` + `df_Reprise` =>> `df_Principals` (concat)

In [ ]:
df_FirstIntention = df_operation1[df_operation1["Name_x"] == "arthroplastie de 1ere intention"]
df_FirstIntention.sample(2)

In [ ]:
df_Reprise = df_operation1[df_operation1["Name_x"] == "arthroplastie de reprise"]
df_Reprise.sample(2)

In [ ]:
df1 = pd.concat([df_FirstIntention,df_Reprise])
df1.shape

In [ ]:
df = df1[["ID_x","Name_x","SegmentAnat"]]

df = df.rename(columns = {"ID_x":"Nb_requests","Name_x":"Categorical_Material","SegmentAnat":"Anatomic_Segment"})
df.head()

In [ ]:
df_Principals = df.groupby(['Categorical_Material','Anatomic_Segment']).count()
df_Principals

### A METTRE DANS TABLEAU : 

In [ ]:
df_PrincipalsV2 = df.groupby(['Anatomic_Segment','Categorical_Material']).count()
df_PrincipalsV2

In [ ]:
df_MaterialOperation2 = df_operationMore[["Id_rq","DMI","SegmentAnat"]]
df_MaterialOperation2.sample(2)

In [ ]:
df_MaterialOperation2["Id_rq"].count()

In [ ]:
df_MaterialOperation2 = df_MaterialOperation2.rename(columns = {"Id_rq":"Nb_requests","DMI":"Categorical_Material","SegmentAnat":"Anatomic Segment"})
df_MaterialOperation2.head()

In [ ]:
df_final = df_MaterialOperation2.groupby(['Anatomic Segment','Categorical_Material']).count()
df_final

In [ ]:
df_MaterialOperation.sample(10)

In [ ]:
df_operation1.sample(2)

In [ ]:
df_operationMore.head(2)

In [ ]:
df_MaterialOperation = df_operationMore[["Id_rq","DMI","SegmentAnat"]]
df_MaterialOperation.sample(5)

In [ ]:
df_final = df_MaterialOperation.groupby(["DMI",'SegmentAnat']).count()
df_final.sample(2)

In [ ]:
df_final["Id_rq"].sum()

In [ ]:
df_final3 = df_MaterialOperation.groupby(['SegmentAnat',"DMI"]).count()
df_final3.sample(2)

#### OUPUT `Operation_Material_Rq_Final.csv`

In [ ]:
df_MaterialOperation.to_csv(path_output+"Operation_Material_Rq_Final.csv", sep=";")

In [ ]:
df_final3.to_csv(path_output+"Nb_requests_by_DMI_by_SegmentAnat.csv")

# <font color='red'>Final csv.files : </font> 

### Operation_Material => DMI 

In [ ]:
df_final.to_csv(path_output+"Operation_Material_Rq.csv")

### Material Labo = details des labos avec les DMI et segments anatomiques

In [ ]:
#Dans le fichier .csv sur excel > j'ai rajouté une colonne département pour pouvoir localiser les labos sur Tableau
df_materialLabo.to_csv(path_output+"SpecialtyAndCity3.csv")

### Principals => main operation "DMI" => focus sur reprise et 1ère intention 

In [ ]:
df_PrincipalsV2.to_csv(path_output+"MainOperation.csv")

---

In [ ]:
#df_SpeMaterial = df_Spe.groupby('Name').count().reset_index()
#df_SpeMaterial.head(2)

In [ ]:
#df_SplitMaterial = df_SpeMaterial[["Name","ID"]]
#df_SplitMaterial = df_SplitMaterial.rename(columns = {"Name":"Categorical_Material","ID" : "Nb_requests"})
#df_SplitMaterial = df_SplitMaterial.sort_values('Nb_requests',ascending=False)
#df_SplitMaterial.head(2)

---

#### Merge  labo & specialty type => majoritairement orthopedique

In [ ]:
df_speType = pd.read_csv(path_input+"Specialty.csv")
df_speType.head()

---

# <font color='tomato'>Annexes </font> 

###### convert to .CSV : `df.to_csv("nom_du_doc.csv")`

In [ ]:
#1ère table : merge des operating specialty avec request : 
df_Spe.to_csv(path_output+"MergeRequestSpecialty.csv")

In [ ]:
#2ème table : nb de demandes par material category : 
df_SplitMaterial.to_csv(path_output+"MaterialSplit.csv")

---

##### `df_nan` => df de request avec uniquement les NaN de `df_request`

In [ ]:
df_nan.head(2)

In [ ]:
df_users = pd.read_csv(path_input+"FosUserUser.csv")
df_users.head()

In [ ]:
df_users.shape

---

#### `df_hi` : Table : Request, HI, Labos, Users

In [ ]:
#df_hi = pd.read_csv(path_output+"Req_HI_Lab_Users.csv")
#df_hi.head(3)